In [1]:
from __future__ import print_function, division
from builtins import range, input

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Reshape,dot,Input,Dense
from keras.models import Sequential,Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

In [4]:
data_dir = Path('/Users/nishantsiv/Desktop/Varsha Pics/exam Paper/Python/ml-100k')

In [5]:
outdir=Path('/Users/nishantsiv/Desktop/Varsha Pics/exam Paper/Python/ml-100k')

In [6]:
def create_data(rating,header_cols):
    data = pd.read_csv(rating,header=None,sep='\t')
    #print(data)
    data.columns = header_cols
    return data

In [7]:
def create_movie_dict(movie_file):
    print(movie_file)
    df = pd.read_csv(movie_file,sep='|', encoding='latin-1',header=None)
    movie_dict = {}
    movie_ids = list(df[0].values)
    movie_name = list(df[1].values)
    for k,v in zip(movie_ids,movie_name):
        movie_dict[k] = v 
    return movie_dict

In [8]:
def train_val(df,val_frac=None):
    X,y = df[['userID','movieID']].values,df['rating'].values
    #Offset the ids by 1 for the ids to start from zero
    X = X - 1 
    if val_frac != None:
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_frac,random_state=0)
        return X_train, X_val, y_train, y_val
    else:
        return X,y


In [9]:
def model(max_users,max_movies,latent_factors):
    user_ID = Input(shape=(1,))
    movie_ID = Input(shape=(1,))
    x = Embedding(max_users,latent_factors, input_length=1)(user_ID)
    print(x.shape)
    y = Embedding(max_movies,latent_factors, input_length=1)(movie_ID)
    print(y.shape)
    out = dot([x,y],axes=2, normalize=False)
    out= Reshape((1,))(out)
    model = Model(inputs=[user_ID,movie_ID],outputs=out)
    print(model.summary())
    return model

In [10]:
train_ratings_df = create_data(f'{data_dir}/u1.base',['userID','movieID','rating','timestamp']) 
test_ratings_df = create_data(f'{data_dir}/u1.test',['userID','movieID','rating','timestamp']) 
X_train, X_val,y_train, y_val = train_val(train_ratings_df,val_frac=0.2)
#X_train, X_val,y_train, y_val = train_frac(train_ratings_df,val_frac=0.2)

movie_dict = create_movie_dict(f'{data_dir}/u.item')
num_users = len(train_ratings_df['userID'].unique())
num_movies = len(train_ratings_df['movieID'].unique())

print(f'Number of users {num_users}')
print(f'Number of items {num_movies}')
model = model(num_users,num_movies,40)
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.compile(loss='mse',optimizer='adam')
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint(f'{outdir}/nn_factor_model.h5', save_best_only=True)]
model.fit([X_train[:,0],X_train[:,1]], y_train, nb_epoch=30, validation_data=([X_val[:,0],X_val[:,1]], y_val), verbose=2, callbacks=callbacks) 

\Users\nishantsiv\Desktop\Varsha Pics\exam Paper\Python\ml-100k/u.item
Number of users 943
Number of items 1650
(None, 1, 40)
(None, 1, 40)
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 40)        37720       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 40) 

TypeError: fit() got an unexpected keyword argument 'nb_epoch'

In [11]:
from keras.models import load_model
#model = load_model(f'{outdir}/nn_factor_model.h5')
X_test,y_test = train_val(test_ratings_df,val_frac=None)
pred = model.predict([X_test[:,0],X_test[:,1]])[:,0]
print('Hold out test set RMSE:',(np.mean((pred - y_test)**2)**0.5))
pred = np.round(pred)
test_ratings_df['predictions'] = pred
test_ratings_df['movie_name'] = test_ratings_df['movieID'].apply(lambda x:movie_dict[x])

Hold out test set RMSE: 3.7192796979344114


In [12]:
test_ratings_df[test_ratings_df['userID'] == 1].sort_values(['rating','predictions'],ascending=False)

,userID,movieID,rating,timestamp,predictions,movie_name
0,1,6,5,887431973,-0.0,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
2,1,12,5,878542960,0.0,"Usual Suspects, The (1995)"
3,1,14,5,874965706,-0.0,"Postino, Il (1994)"
13,1,44,5,878543541,-0.0,Dolores Claiborne (1994)
20,1,60,5,875072370,-0.0,Three Colors: Blue (1993)
...,...,...,...,...,...,...
121,1,243,1,875241390,-0.0,Jungle2Jungle (1997)
126,1,254,1,878541392,-0.0,Batman & Robin (1997)
129,1,259,1,875692979,-0.0,George of the Jungle (1997)
130,1,260,1,875071713,0.0,Event Horizon (1997)
